# Imports

In [ ]:
import re
import geopandas

import pandas as pd
import numpy as np
import bamboolib as bam
import netCDF4 as nc
import xarray as xr
import lightgbm as lgb
import plotly.express as px

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.ticker as mticker
import matplotlib.animation as anim 
import plotly.graph_objects as go 

from time import time
from multiprocessing import Pool
from os import makedirs
from glob import glob
from tqdm.auto import tqdm
from json import dumps, load

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# Constants

In [ ]:
DIR_HAPPI = "/Volumes/T7/ExtremeWeather/Data_WeatherAtHome/HAPPI"

DIR_VCSN = "/Volumes/T7/ExtremeWeather/Data_VCSN"

DIR_PLOTS = "../Plots_Paper"

DIR_PLOTS_THESIS = "../Plots_Thesis"

# Part 1 - Determine how many days are required to reach 25% and 75% total rainfall in the Current-decade (ALL) HAPPI Data

Brief from Luke:

Find the value, N, such that it’s “N wettest days required to reach 25% of climatological annual rainfall” in the current climate/ALL runs, separately for each grid cell. And then repeat to find X such that it’s “X driest days required to also reach 25% of climatological annual rainfall”.

## Processing the data

In [ ]:
def determine_n_days_to_reach_yearly_precip_percent(
    df: pd.DataFrame,
    grid_cell_dict: dict(),
    target_wet_percent: float,
    target_dry_percent: float,
    target_col: str,
    model_name: str,
    region_name: str
) -> (list, list):
    for grid_cell in df["grid_cell"].unique():
        df_cell = df.loc[df["grid_cell"] == grid_cell]
        df_cell = df_cell.sort_values([target_col], ascending=False).reset_index(drop=True)

        total_target = df_cell[target_col].sum()

        df_cell[f"cumulative_{target_col}"] = df_cell[target_col].cumsum()
        df_cell[f"cumulative_{target_col}_percent"] = df_cell[f"cumulative_{target_col}"] / total_target

        df_cell["n_wet"] = df_cell[f"cumulative_{target_col}_percent"] >= target_wet_percent
        df_cell["n_dry"] = df_cell[f"cumulative_{target_col}_percent"] >= target_dry_percent
        
        grid_cell_dict["region"].append(region_name)
        grid_cell_dict["grid_cell"].append(grid_cell)
        grid_cell_dict["model"].append(model_name)
        grid_cell_dict["wet"].append(int(df_cell[df_cell["n_wet"]].index.min()))
        grid_cell_dict["dry"].append(int(df_cell.shape[0] - df_cell[df_cell["n_dry"]].index.min()))


def generate_fixed_precip_percent_chunk_wah(file: str) -> None:
    try:
        region = re.findall(r"region=(.*?)/", file)[0]
        source = re.findall(r"sim=(.*?)/", file)[0]
        year = "" if "year" not in file else re.findall(r"year=(.*?)/", file)[0]

        grid_cell_dict = {
            "region": [],
            "grid_cell": [],
            "model": [],
            "wet": [],
            "dry": []
        }

        target_wet_percent = 0.25
        target_dry_percent = 0.75

        target_col = "precip_mm"
        input_cols = [
            "grid_cell",
            target_col
        ]

        for model_dir in sorted(glob(f"{file}model_tag=*/")):
            model_name = re.findall(r"model_tag=(.*?)/", model_dir)[0]
            
            determine_n_days_to_reach_yearly_precip_percent(
                df=pd.read_parquet(model_dir, engine="pyarrow", columns=input_cols),
                grid_cell_dict=grid_cell_dict,
                target_wet_percent=target_wet_percent,
                target_dry_percent=target_dry_percent,
                target_col=target_col,
                model_name=model_name,
                region_name=region
            )
        
        parent_dir = f"{DATA_DIR_SIM_FIXED_PRECIP_OUTPUT}/year={year}/source={source}"
        makedirs(parent_dir, exist_ok=True)
        file_path = f"{parent_dir}/{region}.parquet"
        df = pd.DataFrame(data=grid_cell_dict)
        df.to_parquet(file_path)
        
    except Exception as e:
        print(f"Encountered an error: {e}")

In [ ]:
def process_fixed_precip_percent_wah(sim_type: str, files: [str]) -> None:
    print(f"\nFor {sim_type} forcing I found {len(files):,} files, opening the pool [ size: {POOL_SIZE} ]")

    time_start = time()
    
    # ===========================================
    # Pooling is broken in notebooks, I can't be bothered figuring this out so just run it from an IDE or terminal (it will take around 1 minute)
    # pool = Pool(processes=POOL_SIZE)
    # for _ in tqdm(pool.imap_unordered(generate_fixed_precip_percent_chunk_wah, files), total=len(files)):
    #     pass    # We just need some dummy statement for the progress bar
    # pool.close()
    
    # ============================================
    # Slow single threaded version
    # If you really want to run in this notebook then do this one (it will take around 10 minutes)
    for file in tqdm(files, desc="Slow single threaded loop"):
        generate_fixed_precip_percent_chunk_wah(file)

    print(f"Finished processing all {len(files):,} files. The pool has been closed.")
    print(f"The whole thing took: {(time() - time_start):,.2f} seconds")

In [ ]:
POOL_SIZE = 10

In [ ]:
DATA_DIR_SIM_INPUT = f"{DIR_HAPPI}/Processed/"
DATA_DIR_SIM_FIXED_PRECIP_OUTPUT = f"{DIR_HAPPI}/RVI/Chunks_Fixed_PrecipPercent/"

In [ ]:
process_fixed_precip_percent_wah("ALL", sorted(glob(f"{DATA_DIR_SIM_INPUT}/year=*/region=*/sim=ALL/")))

# Part 2 - Determine how much rain falls within fixed number of days for each grid cell of each model of each year

Once you’ve identified N and X for each gridcell, keep those numbers fixed from now on. And then search through each ensemble member to just calculate the total millimetres of rain which fell on the same fixed N wettest days and X driest days for each model year. That way, you’ll produce a distribution of answers (one model year each), with the units being how many mm of rain fell on the N wettest days and X driest days, at each location.

In [ ]:
df_ = df.loc[df["grid_cell"] == "-35.995018_174.70364"]
df_

## Figure out what good N and X values are for the various grid_cells

In [ ]:
df = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_PrecipPercent/")

In [ ]:
group_cols = [
    "region",
    "grid_cell",
    # "year",
    "source"
]

target_cols = [
    "wet",
    "dry"
]

In [ ]:
df_grouped = df[group_cols + target_cols].groupby(by=group_cols).mean()

df_grouped = df_grouped.dropna().reset_index().round()

df_grouped

In [ ]:
grid_cell_map = dict()

for row in df_grouped.loc[df_grouped["source"] == "ALL"].copy()[["grid_cell", "wet", "dry"]].values:
    grid_cell_map[row[0]] = {
        "wet": row[1],
        "dry": row[2]
    }
    
file_path = f"{DIR_HAPPI}/RVI/fixed_precip_grid_cell_map.json"
with open(file_path, "w") as f_out:
    f_out.write(dumps(grid_cell_map))

## Processing the data

In [ ]:
def determine_total_precip_for_each_grid_cell_on_specific_days(
    df: pd.DataFrame,
    grid_cell_dict: dict,
    grid_cell_day_map: dict,
    target_col: str,
    model_name: str,
    region_name: str
) -> (list, list):
    for grid_cell in df["grid_cell"].unique():
        df_cell = df.loc[df["grid_cell"] == grid_cell]
        df_cell = df_cell.sort_values([target_col], ascending=False).reset_index(drop=True)
        
        target_wet_days = int(grid_cell_day_map[grid_cell]["wet"])
        target_dry_days = df_cell.shape[0] - int(grid_cell_day_map[grid_cell]["dry"])
        
        total_precip_wet_days = df_cell.iloc[0:target_wet_days][target_col].sum()
        total_precip_dry_days = df_cell.iloc[target_dry_days:][target_col].sum()
        
        grid_cell_dict["region"].append(region_name)
        grid_cell_dict["grid_cell"].append(grid_cell)
        grid_cell_dict["model"].append(model_name)
        grid_cell_dict["total_precip_wet_days"].append(total_precip_wet_days)
        grid_cell_dict["total_precip_dry_days"].append(total_precip_dry_days)


def generate_fixed_days_chunk_wah(file: str) -> None:
    try:
        region = re.findall(r"region=(.*?)/", file)[0]
        source = re.findall(r"sim=(.*?)/", file)[0]
        year = "" if "year" not in file else re.findall(r"year=(.*?)/", file)[0]

        grid_cell_dict = {
            "region": [],
            "grid_cell": [],
            "model": [],
            "total_precip_wet_days": [],
            "total_precip_dry_days": []
        }
        
        grid_cell_day_map = load(open(f"{DIR_HAPPI}/RVI/fixed_precip_grid_cell_map.json"))

        target_col = "precip_mm"
        input_cols = [
            "grid_cell",
            target_col
        ]

        for model_dir in sorted(glob(f"{file}model_tag=*/")):
            model_name = re.findall(r"model_tag=(.*?)/", model_dir)[0]
            
            determine_total_precip_for_each_grid_cell_on_specific_days(
                df=pd.read_parquet(model_dir, engine="pyarrow", columns=input_cols),
                grid_cell_dict=grid_cell_dict,
                grid_cell_day_map=grid_cell_day_map,
                target_col=target_col,
                model_name=model_name,
                region_name=region
            )
        
        parent_dir = f"{DATA_DIR_SIM_FIXED_DAYS_OUTPUT}/year={year}/source={source}"
        makedirs(parent_dir, exist_ok=True)
        file_path = f"{parent_dir}/{region}.parquet"
        df = pd.DataFrame(data=grid_cell_dict)
        df.to_parquet(file_path)
        
    except Exception as e:
        print(f"Encountered an error: {e}")

In [ ]:
def process_fixed_days_wah(sim_type: str, files: [str]) -> None:
    print(f"\nFor {sim_type} forcing I found {len(files):,} files, opening the pool [ size: {POOL_SIZE} ]")

    time_start = time()
    
    # ===========================================
    # Pooling is broken in notebooks, I can't be bothered figuring this out so just run it from an IDE or terminal (it will take around 1 minute)
    # pool = Pool(processes=POOL_SIZE)
    # for _ in tqdm(pool.imap_unordered(generate_fixed_days_chunk_wah, files), total=len(files)):
    #     pass    # We just need some dummy statement for the progress bar
    # pool.close()
    
    # ============================================
    # Slow single threaded version
    # If you really want to run in this notebook then do this one (it will take around 10 minutes)
    for file in tqdm(files, desc="Slow single threaded loop"):
        generate_fixed_days_chunk_wah(file)

    print(f"Finished processing all {len(files):,} files. The pool has been closed.")
    print(f"The whole thing took: {(time() - time_start):,.2f} seconds")

In [ ]:
DATA_DIR_SIM_INPUT = f"{DIR_HAPPI}/Processed/"
DATA_DIR_SIM_FIXED_DAYS_OUTPUT = f"{DIR_HAPPI}/RVI/Chunks_Fixed_Days/"

In [ ]:
process_fixed_days_wah("ALL", sorted(glob(f"{DATA_DIR_SIM_INPUT}/year=*/region=*/sim=ALL/")))
process_fixed_days_wah("NAT", sorted(glob(f"{DATA_DIR_SIM_INPUT}/year=*/region=*/sim=NAT/")))

# Part 3 - Repeat part 2 for the 3-DEG (HOT) HAPPI data

Repeat for your other warmer-world experiment (for the paper, this will be the 3 degree HAPPI simulations), again keeping N and X fixed as the values defined based on the current climate/ALL experiments.

In [ ]:
DATA_DIR_SIM_INPUT = f"{DIR_HAPPI}/Processed/"

DATA_DIR_SIM_FIXED_DAYS_OUTPUT = f"{DIR_HAPPI}/RVI/Chunks_Fixed_Days"

In [ ]:
process_fixed_days_wah("HOT", sorted(glob(f"{DATA_DIR_SIM_INPUT}/year=*/region=*/sim=HOT/")))

# Part 4 - Produce ensemble maps showing differences

Now you have two distributions (one current climate, one warmer world) of “mm which fell on the N wettest days of the year”, and then another two distributions (one current climate, one warmer world) of “mm which fell on X driest days of the year”. 

Then you can produce a map of the ensemble median change (percent difference) in rain which falls on the N wettest days of the year, and on the X driest days of the year – this is what we already have (see attached figures). But what we also want to do is produce equivalent maps but comparing the wetter ensemble members of each experiment. 

So we want to look at the 95th percentile ensemble member answer in the current climate and compare with the 95th percentile ensemble member answer in the future climate. So rather than “an additional 8% of rain falls in the wettest N days of the year”, now it might be “an additional 12% of rain falls in the wettest N days of the year when it’s an extreme wet year”. Or something. And then vice-versa, look to compare the 5th percentile of ensemble members in the future against the 5th percentile of ensemble members in current climate for the “mm falling on X driest days” metric, too. Example for one grid cell in Northland is attached.

## Read the data

In [ ]:
sel = [("region", "!=", "Area Outside Region")]

df = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_Days/", filters=sel)

In [ ]:
df_ALL = df.loc[df["source"] == "ALL"].copy().reset_index(drop=True)
df_HOT = df.loc[df["source"] == "HOT"].copy().reset_index(drop=True)

In [ ]:
# Attach days required to datasets
with open(f"{DIR_HAPPI}/RVI/fixed_precip_grid_cell_map.json") as json_file:
    fixed_precip_grid_cell_map = load(json_file)
    
fixed_precip_grid_cell_map_wet = {}
fixed_precip_grid_cell_map_dry = {}

for key, value in fixed_precip_grid_cell_map.items():
    fixed_precip_grid_cell_map_wet[key] = int(value["wet"])
    fixed_precip_grid_cell_map_dry[key] = int(value["dry"])

df_ALL["days_wet"] = df_ALL["grid_cell"].map(fixed_precip_grid_cell_map_wet)
df_ALL["days_dry"] = df_ALL["grid_cell"].map(fixed_precip_grid_cell_map_dry)
df_HOT["days_wet"] = df_HOT["grid_cell"].map(fixed_precip_grid_cell_map_wet)
df_HOT["days_dry"] = df_HOT["grid_cell"].map(fixed_precip_grid_cell_map_dry)

## Recreate the median difference maps

### Manually 

In [ ]:
group_cols = [
    "grid_cell",
]

target_cols = [
    "total_precip_wet_days",
    "total_precip_dry_days",
]

df_ALL_grouped = df_ALL[group_cols + target_cols].groupby(by=group_cols).median().reset_index()
df_HOT_grouped = df_HOT[group_cols + target_cols].groupby(by=group_cols).median().reset_index()

df_merged = df_HOT_grouped.merge(
    right=df_ALL_grouped,
    how="left",
    on="grid_cell",
    suffixes=["_HOT", "_ALL"]
)

df_merged["total_precip_wet_days_diff"] = df_merged["total_precip_wet_days_HOT"] / df_merged["total_precip_wet_days_ALL"]
df_merged["total_precip_dry_days_diff"] = df_merged["total_precip_dry_days_HOT"] / df_merged["total_precip_dry_days_ALL"]

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_merged["grid_cell"].str.split('_', n=2, expand=True)
split_df.columns = ["latitude", "longitude"]
df_merged = pd.merge(df_merged, split_df, how="left", left_index=True, right_index=True)
df_merged["longitude"] = df_merged["longitude"].astype(float)
df_merged["latitude"] = df_merged["latitude"].astype(float)

df_merged

In [ ]:
DEFAULT_CRS = "EPSG:4326"

df_merged = df_merged.set_geometry(
    geopandas.points_from_xy(
            x=df_merged["longitude"], 
            y=df_merged["latitude"], 
            crs=DEFAULT_CRS
    )
)

df_merged

In [ ]:
fig = px.scatter(
    df_merged, 
    x='longitude', 
    y='latitude', 
    color='total_precip_wet_days_diff', 
    symbol_sequence=['circle'], 
    height=800, 
    width=800,
    template="plotly_white",
    color_continuous_scale="GnBu",
    # range_color=(100, 130)
)

fig.update_traces(marker={'size': 19})

fig

In [ ]:
fig = px.scatter(
    df_merged, 
    x='longitude', 
    y='latitude', 
    color='total_precip_dry_days_diff', 
    # symbol_sequence=['diamond'], 
    symbol_sequence=['circle'], 
    height=800, 
    width=800,
    template="plotly_white",
    color_continuous_scale="OrRd",
    # range_color=(100, 130)
)

fig.update_traces(marker={'size': 19})

fig

In [ ]:
df_ALL

In [ ]:
dfs = []

# Iterate over each grid cell
for grid_cell in tqdm(df_ALL["grid_cell"].unique(), desc="Grid_cell Loop"):
    # Filter to that grid cell data
    df_temp_ALL = df_ALL.loc[df_ALL["grid_cell"] == grid_cell].copy()
    df_temp_HOT = df_HOT.loc[df_HOT["grid_cell"] == grid_cell].copy()
    
    # Determine wet and dry quantiles
    df_temp_ALL["wet_quantile"] = pd.qcut(df_temp_ALL["total_precip_wet_days"], q=100, labels=np.arange(1, 101, 1))
    df_temp_ALL["dry_quantile"] = pd.qcut(df_temp_ALL["total_precip_dry_days"], q=100, labels=np.arange(1, 101, 1))
    df_temp_HOT["wet_quantile"] = pd.qcut(df_temp_HOT["total_precip_wet_days"], q=100, labels=np.arange(1, 101, 1))
    df_temp_HOT["dry_quantile"] = pd.qcut(df_temp_HOT["total_precip_dry_days"], q=100, labels=np.arange(1, 101, 1))
    
    # Change the quantile cols to strings then integers as they start as objects
    df_temp_ALL["wet_quantile"] = df_temp_ALL["wet_quantile"].astype(str).astype(int)
    df_temp_ALL["dry_quantile"] = df_temp_ALL["dry_quantile"].astype(str).astype(int)
    df_temp_HOT["wet_quantile"] = df_temp_HOT["wet_quantile"].astype(str).astype(int)
    df_temp_HOT["dry_quantile"] = df_temp_HOT["dry_quantile"].astype(str).astype(int)

    # ==================================================================================================================================
    # I am quite tired and seem to have broken the "merge" function. I can't be bothered fixing it so this is a fast workaround
    
    # Group by the quantiles taking the median of each quantile, this "bins them"
    group_cols = [
        "grid_cell",
        "days_wet",
        "days_dry",
    ]
    wet_cols = [
        "total_precip_wet_days",
        "wet_quantile",
    ]
    dry_cols = [
        "total_precip_dry_days",
        "dry_quantile"
    ]
    df_ALL_wet = df_temp_ALL[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
    df_ALL_dry = df_temp_ALL[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
    df_ALL_wet["total_precip_dry_days"] = df_ALL_dry["total_precip_dry_days"]
    df_ALL_wet["dry_quantile"] = df_ALL_dry["dry_quantile"]
    df_HOT_wet = df_temp_HOT[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
    df_HOT_dry = df_temp_HOT[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
    df_HOT_wet["total_precip_dry_days"] = df_HOT_dry["total_precip_dry_days"]
    df_HOT_wet["dry_quantile"] = df_HOT_dry["dry_quantile"]

    # Merge them back into one
    df_merged = df_ALL_wet[group_cols]
    for col in [
        "total_precip_wet_days",
        "total_precip_dry_days",
        "wet_quantile",
        "dry_quantile",
    ]:
        if "wet" in col:
            df_merged[f"{col}_HOT"] = df_HOT_wet[col]
            df_merged[f"{col}_ALL"] = df_ALL_wet[col]
        else:
            df_merged[f"{col}_HOT"] = df_HOT_dry[col]
            df_merged[f"{col}_ALL"] = df_ALL_dry[col]
    
    # ==================================================================================================================================
    
    # Determine total_precip_xxx_days_fraction here to save time
    df_merged["total_precip_wet_days_fraction"] = df_merged["total_precip_wet_days_HOT"] / df_merged["total_precip_wet_days_ALL"]
    df_merged["total_precip_dry_days_fraction"] = df_merged["total_precip_dry_days_HOT"] / df_merged["total_precip_dry_days_ALL"]
    
    # Pull out the lon and lat values
    grid_cell_parts = grid_cell.split("_")
    lon = grid_cell_parts[0]
    lat = grid_cell_parts[1]
    df_merged["longitude"] = lon
    df_merged["latitude"] = lat
    
    # Append this to the big list
    dfs.append(df_merged)

# Concat all the dfs into one giga-one
df_merged = pd.concat(dfs)

### Automated

In [ ]:
def determine_quantiles_for_data(df_ALL, df_HOT, num_quantiles=100):
    dfs = []

    # Iterate over each grid cell
    for grid_cell in tqdm(df_ALL["grid_cell"].unique(), desc="Grid_cell Loop"):
        # Filter to that grid cell data
        df_temp_ALL = df_ALL.loc[df_ALL["grid_cell"] == grid_cell].copy()
        df_temp_HOT = df_HOT.loc[df_HOT["grid_cell"] == grid_cell].copy()

        # Determine wet and dry quantiles
        df_temp_ALL["wet_quantile"] = pd.qcut(df_temp_ALL["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_ALL["dry_quantile"] = pd.qcut(df_temp_ALL["total_precip_dry_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_HOT["wet_quantile"] = pd.qcut(df_temp_HOT["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_HOT["dry_quantile"] = pd.qcut(df_temp_HOT["total_precip_dry_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))

        # Change the quantile cols to strings then integers as they start as objects
        df_temp_ALL["wet_quantile"] = df_temp_ALL["wet_quantile"].astype(str).astype(int)
        df_temp_ALL["dry_quantile"] = df_temp_ALL["dry_quantile"].astype(str).astype(int)
        df_temp_HOT["wet_quantile"] = df_temp_HOT["wet_quantile"].astype(str).astype(int)
        df_temp_HOT["dry_quantile"] = df_temp_HOT["dry_quantile"].astype(str).astype(int)

        # ==================================================================================================================================
        # I am quite tired and seem to have broken the "merge" function. I can't be bothered fixing it so this is a fast workaround

        # Group by the quantiles taking the median of each quantile, this "bins them"
        group_cols = [
            "grid_cell",
            "days_wet",
            "days_dry",
        ]
        wet_cols = [
            "total_precip_wet_days",
            "wet_quantile",
        ]
        dry_cols = [
            "total_precip_dry_days",
            "dry_quantile"
        ]
        df_ALL_wet = df_temp_ALL[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        df_ALL_dry = df_temp_ALL[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
        df_ALL_wet["total_precip_dry_days"] = df_ALL_dry["total_precip_dry_days"]
        df_ALL_wet["dry_quantile"] = df_ALL_dry["dry_quantile"]
        df_HOT_wet = df_temp_HOT[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        df_HOT_dry = df_temp_HOT[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
        df_HOT_wet["total_precip_dry_days"] = df_HOT_dry["total_precip_dry_days"]
        df_HOT_wet["dry_quantile"] = df_HOT_dry["dry_quantile"]

        # Merge them back into one
        df_merged = df_ALL_wet[group_cols]
        for col in [
            "total_precip_wet_days",
            "total_precip_dry_days",
            "wet_quantile",
            "dry_quantile",
        ]:
            if "wet" in col:
                df_merged[f"{col}_HOT"] = df_HOT_wet[col]
                df_merged[f"{col}_ALL"] = df_ALL_wet[col]
            else:
                df_merged[f"{col}_HOT"] = df_HOT_dry[col]
                df_merged[f"{col}_ALL"] = df_ALL_dry[col]

        # ==================================================================================================================================

        # Determine total_precip_xxx_days_fraction here to save time
        df_merged["total_precip_wet_days_fraction"] = (df_merged["total_precip_wet_days_HOT"] / df_merged["total_precip_wet_days_ALL"]).astype(float).round(3)
        df_merged["total_precip_dry_days_fraction"] = (df_merged["total_precip_dry_days_HOT"] / df_merged["total_precip_dry_days_ALL"]).astype(float).round(3)
        
        
        # Pull out the lon and lat values
        grid_cell_parts = grid_cell.split("_")
        lat = float(grid_cell_parts[0])
        lon = float(grid_cell_parts[1])
        df_merged["longitude"] = lon
        df_merged["latitude"] = lat

        # Append this to the big list
        dfs.append(df_merged)

    # Concat all the dfs into one giga-one
    return pd.concat(dfs)
    


def plot_difference_map_quantile(df_merged, target_quantile, target_scenario):
    colour_map = {
        "Dry End": "OrRd",
        "Wet End": "GnBu"
    }
    
    fig = px.scatter(
        df_merged, 
        x='longitude', 
        y='latitude', 
        color=f"{target_scenario}", 
        symbol_sequence=["circle"], 
        height=900, 
        width=800,
        template="plotly_white",
        color_continuous_scale=colour_map[target_scenario],
        range_color=(0.75, 1.25)
    )
    
    fig.update_layout(
        title=f"Change in Expected Precipitation (HOT / ALL) {target_scenario} [ {target_quantile}th bin ]",
        coloraxis_colorbar=dict(title="")
    )
    
    fig.update_xaxes(title="Longitude", visible=True)
    fig.update_yaxes(title="Latitude", visible=True)
    
    fig.update_traces(marker={'size': 21})

    return fig

In [ ]:
df_merged = determine_quantiles_for_data(df_ALL, df_HOT, 100)

In [ ]:
df_merged.to_parquet(f"{DIR_HAPPI}/RVI/fixed_day_grid_cell_quantiles.parquet")

In [ ]:
df_merged.rename(
    columns={
        "total_precip_wet_days_fraction": "Wet End", 
        "total_precip_dry_days_fraction": "Dry End"
    }, 
    inplace=True
)

In [ ]:
target_quantile = 50
df_quantile = df_merged.loc[df_merged["dry_quantile_ALL"] == target_quantile].copy()

fig_wet = plot_difference_map_quantile(df_quantile, target_quantile, "Wet End")
fig_dry = plot_difference_map_quantile(df_quantile, target_quantile, "Dry End")

fig_wet.show()
fig_dry.show()

fig_wet.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.html")
fig_dry.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.html")

fig_wet.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.png", format="png", scale=2)
fig_dry.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.png", format="png", scale=2)

In [ ]:
target_quantile = 10
df_quantile = df_merged.loc[df_merged["dry_quantile_ALL"] == target_quantile].copy()

fig_wet = plot_difference_map_quantile(df_quantile, target_quantile, "Wet End")
fig_dry = plot_difference_map_quantile(df_quantile, target_quantile, "Dry End")

fig_wet.show()
fig_dry.show()

fig_wet.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.html")
fig_dry.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.html")

fig_wet.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.png", format="png", scale=2)
fig_dry.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.png", format="png", scale=2)

In [ ]:
target_quantile = 90
df_quantile = df_merged.loc[df_merged["dry_quantile_ALL"] == target_quantile].copy()

fig_wet = plot_difference_map_quantile(df_quantile, target_quantile, "Wet End")
fig_dry = plot_difference_map_quantile(df_quantile, target_quantile, "Dry End")

fig_wet.show()
fig_dry.show()

fig_wet.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.html")
fig_dry.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.html")

fig_wet.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.png", format="png", scale=2)
fig_dry.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.png", format="png", scale=2)

### For fun, do a differences of differences because I am lazy

In [ ]:
def plot_difference_of_differences_map(df_A, df_B, target_quantile_A, target_quantile_B, target_scenario):
    colour_map = {
        "dry_days": "OrRd",
        "wet_days": "GnBu"
    }
    
    target_col = f"total_precip_{target_scenario}"
    df_merged = df_A.copy()
    df_merged.rename(
        columns={
            f"{target_col}_ALL": f"{target_col}_ALL_A",
            f"{target_col}_HOT": f"{target_col}_HOT_A",
            f"{target_col}_fraction": f"{target_col}_fraction_A"
        }, 
        inplace=True
    )
    df_merged[f"{target_col}_ALL_B"] = df_B[f"{target_col}_ALL"]
    df_merged[f"{target_col}_HOT_B"] = df_B[f"{target_col}_HOT"]
    df_merged[f"{target_col}_fraction_B"] = df_B[f"{target_col}_fraction"]
    
    df_merged[f"{target_col}_HOT_diff"] = df_merged[f"{target_col}_HOT_A"] - df_merged[f"{target_col}_HOT_B"]
    df_merged[f"{target_col}_ALL_diff"] = df_merged[f"{target_col}_ALL_A"] - df_merged[f"{target_col}_ALL_B"]
    
    df_merged[f"{target_col}_fraction"] = (df_merged[f"{target_col}_HOT_diff"] / df_merged[f"{target_col}_ALL_diff"]).astype(float).round(2)
    df_merged[f"{target_col}_fraction_diff"] = (df_merged[f"{target_col}_fraction_A"] - df_merged[f"{target_col}_fraction_B"]).astype(float).round(2)
    
    fig = px.scatter(
        df_merged, 
        x='longitude', 
        y='latitude', 
        color=f"total_precip_{target_scenario}_fraction_diff", 
        symbol_sequence=["circle"], 
        height=900, 
        width=900,
        template="plotly_white",
        color_continuous_scale=colour_map[target_scenario],
        # range_color=(-0.05, 0.10)
    )
    
    fig.update_layout(
        title=f"Fraction Precipitation Difference (HOT / ALL) {target_scenario.replace('_', ' ')} [ {target_quantile_A}th quantile - {target_quantile_B}th quantile]",
        yaxis_title=None,
        xaxis_title=None
    )
    
    # fig.update_xaxes(visible=False)
    # fig.update_yaxes(visible=False)
    
    fig.update_traces(marker={'size': 21})

    return fig

In [ ]:
quantile_A = 90
quantile_B = 50

df_A = df_merged.loc[df_merged["dry_quantile_ALL"] == quantile_A].copy().reset_index(drop=True)
df_B = df_merged.loc[df_merged["dry_quantile_ALL"] == quantile_B].copy().reset_index(drop=True)

fig_wet = plot_difference_of_differences_map(df_A, df_B, quantile_A, quantile_B, "wet_days")
fig_dry = plot_difference_of_differences_map(df_A, df_B, quantile_A, quantile_B, "dry_days")

fig_wet.show()
fig_dry.show()

### Luke made some histograms for just Northland, recreate those

In [ ]:
df_northland = df.loc[(df["region"] == "Northland Region") & (df["grid_cell"] == "-35.995018_174.70364")].copy().reset_index(drop=True)

df_northland["days_wet"] = df_northland["grid_cell"].map(fixed_precip_grid_cell_map_wet)
df_northland["days_dry"] = df_northland["grid_cell"].map(fixed_precip_grid_cell_map_dry)

df_northland

In [ ]:
df_merged_ = df_merged.loc[df_merged["grid_cell"] == "-35.995018_174.70364"]
df_merged_

In [ ]:
fig = px.histogram(
    df_northland, 
    x='total_precip_dry_days', 
    color='source', 
    barmode='overlay', 
    nbins=100, 
    height=600, 
    width=1000, 
    range_y=(0, 300), 
    range_x=(0, 600), 
    opacity=0.7,
    template="plotly_white",
    title="Northland - driest 322 days"
)

fig.add_shape(type='line', x0=179, x1=179, y0=0, y1=250, opacity=0.8, line=dict(color='green', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=202, x1=202, y0=0, y1=250, opacity=0.8, line=dict(color='green', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=243, x1=243, y0=0, y1=250, opacity=0.8, line=dict(color='brown', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=271, x1=271, y0=0, y1=250, opacity=0.8, line=dict(color='brown', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=265, x1=265, y0=0, y1=300, opacity=0.8, line=dict(color='black', width=6, dash='solid'), xref='x', yref='y', layer='above')

fig.update_traces(marker_line_width=1,marker_line_color="black")

fig


In [ ]:
fig = px.histogram(
    df_northland, 
    x='total_precip_wet_days', 
    color='source', 
    barmode='overlay', 
    nbins=100, 
    height=600, 
    width=1000, 
    range_y=(0, 300), 
    range_x=(0, 600), 
    opacity=0.7,
    template="plotly_white",
    title="Northland - wettest 5 days"
)

fig.add_shape(type='line', x0=269, x1=269, y0=0, y1=250, opacity=0.9, line=dict(color='green', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=296, x1=296, y0=0, y1=250, opacity=0.9, line=dict(color='green', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=369, x1=369, y0=0, y1=250, opacity=0.9, line=dict(color='purple', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=415, x1=415, y0=0, y1=250, opacity=0.9, line=dict(color='purple', width=4, dash='solid'), xref='x', yref='y', layer='above')
fig.add_shape(type='line', x0=265, x1=265, y0=0, y1=300, opacity=0.9, line=dict(color='black', width=6, dash='solid'), xref='x', yref='y', layer='above')

fig.update_traces(marker_line_width=1,marker_line_color="black")

fig

## Comparing percentiles - Grid-Cell

In [ ]:
def grid_cell_determine_model_quantiles(df: pd.DataFrame, target_col: str):
    # Step: Sort column(s) total_precip_wet_days ascending (A-Z)
    df = df.sort_values(by=[target_col], ascending=[True])

    # Step: Split into ALL and HOT
    df_ALL = df.loc[df["source"] == "ALL"]
    df_HOT = df.loc[df["source"] == "HOT"]

    # Step: Bin these based on wet precip column
    df_ALL["quantile"] = pd.qcut(df_ALL[target_col], q=100, labels=np.arange(1, 101, 1))
    df_HOT["quantile"] = pd.qcut(df_HOT[target_col], q=100, labels=np.arange(1, 101, 1))
    
    # Step: Group them by quantile
    cols = ["quantile", target_col]
    df_ALL = df_ALL[cols].groupby(by=["quantile"]).median().reset_index(drop=False)
    df_HOT = df_HOT[cols].groupby(by=["quantile"]).median().reset_index(drop=False)
    
    # Step: Merge them into one
    df_merged = df_HOT.merge(
        right=df_ALL,
        on=["quantile"],
        suffixes=["_HOT", "_ALL"]
    )
    
    # Step: Determine differences
    df_merged["difference"] = df_merged[f"{target_col}_HOT"] / df_merged[f"{target_col}_ALL"]
    
    return df_merged



def grid_cell_plot_quantile_difference(df:pd.DataFrame, target_col: str, grid_cell: str, region_name: str, num_days: str):
    colour_map = {
        "total_precip_wet_days": "rgba(200, 0, 150, 0.8)",
        "total_precip_dry_days": "rgba(200, 100, 0, 0.8)"
    }
    
    fig = px.line(
        df.sort_values(by=['quantile'], ascending=[True]), 
        x='quantile', 
        y='difference', 
        template='plotly_white', 
        range_y=(0.8, 1.2), 
        height=400,
        width=700,
        title=f"Grid_cell: {grid_cell} [ {region_name} ]"
    )

    fig.update_traces(mode="lines+markers", line_color=colour_map[target_col])
    
    fig.update_yaxes(title=f"{target_col.replace('_', ' ').upper()} {num_days} days")
    fig.update_xaxes(title="Ranked W@H Simulation (percentiles)")
    
    return fig

In [ ]:
print(f"There are {len(df['grid_cell'].unique())} unique grid-cells")

grid_cell_day_map = load(open(f"{DIR_HAPPI}/RVI/fixed_precip_grid_cell_map.json"))

In [ ]:
region_name = "Northland Region"

for grid_cell in tqdm(df.loc[df["region"] == region_name]["grid_cell"].unique()[:1]):
    df_single_grid = df.loc[df["grid_cell"] == grid_cell]
    day_map_result = grid_cell_day_map[grid_cell]
    
    target_col = "total_precip_wet_days"
    df_quantiles = grid_cell_determine_model_quantiles(df_single_grid, target_col)
    grid_cell_plot_quantile_difference(df_quantiles, target_col, grid_cell, region_name, int(day_map_result["wet"])).show()
    
    target_col = "total_precip_dry_days"
    df_quantiles = grid_cell_determine_model_quantiles(df_single_grid, target_col)
    grid_cell_plot_quantile_difference(df_quantiles, target_col, grid_cell, region_name, int(day_map_result["dry"])).show()    

In [ ]:
df

## Comparing Percentiles - Total Yearly Precipitation

In [ ]:
def write_json_to_disk(data, parent_dir, file_name):
    makedirs(parent_dir, exist_ok=True)
    file_path = f"{parent_dir}/{file_name}.json"
    with open(file_path, "w") as f_out:
        f_out.write(dumps(data))

### Read in the daily precip data and group into grid_cell and model_tag

In [ ]:
sel = [("region", "!=", "Area Outside Region")]

df = pd.read_parquet(f"{DIR_HAPPI}/Processed/", filters=sel)

In [ ]:
df

In [ ]:
group_cols = [
    "sim",
    "model_tag",
    "region",
    "grid_cell"
]

target_col = "precip_mm"

df_grouped = df[group_cols + [target_col]].groupby(by=group_cols).sum().reset_index()

df_grouped = df_grouped.loc[df_grouped[target_col] > 0].reset_index(drop=True)

for col in tqdm(group_cols):
    df_grouped[col] = df_grouped[col].astype(str)

df_grouped

### Quickly grab the climatology expected mean / median precip to use later

In [ ]:
df_climatology = df_grouped.loc[df_grouped["sim"] == "ALL"].copy().reset_index(drop=True)

climatology_precip_mean = df_climatology[target_col].mean()
climatology_precip_median = df_climatology[target_col].median()

In [ ]:
climatology_precip_mean

In [ ]:
climatology_precip_median

### Determine expected regional annual precipitations

In [ ]:
group_cols = [
    "sim",
    "region"
]

df_regions = df_grouped[group_cols + [target_col]].groupby(by=group_cols).median().reset_index()

expected_precip_map_region = {}

for i in range(0, df_regions.shape[0]):
    row = df_regions.iloc[i]
    
    key_region = str(row["region"])
    key_sim = str(row["sim"])
    value_precip = round(float(row[target_col]), 2)
    
    if key_region not in expected_precip_map_region:
        expected_precip_map_region[key_region] = {"ALL": value_precip, "HOT": 0.0}
    else:
        expected_precip_map_region[key_region][key_sim] = value_precip

expected_precip_map_region

In [ ]:
write_json_to_disk(expected_precip_map_region, DIR_HAPPI, "expected_annual_rainfall_region_map")

### Determine model rankings (quantiles) - region based

In [ ]:
regional_model_quantiles_ALL = {}
regional_model_quantiles_HOT = {}

regional_model_quantiles = {
    # ALL
    "Percentile 05 - ALL": {},
    "Percentile 10 - ALL": {},
    "Percentile 50 - ALL": {},
    "Percentile 90 - ALL": {},
    "Percentile 95 - ALL": {},
    # HOT
    "Percentile 05 - HOT": {},
    "Percentile 10 - HOT": {},
    "Percentile 50 - HOT": {},
    "Percentile 90 - HOT": {},
    "Percentile 95 - HOT": {},
}

group_cols = [
    "sim",
    "model_tag",
    "region"
]
df_models = df_grouped[group_cols + [target_col]].groupby(by=group_cols).mean().reset_index()

for region in tqdm(df_models["region"].unique(), desc="Region Loop", leave=1):
    df_region = df_models.loc[df_models["region"] == region].copy().sort_values(by=[target_col]).reset_index(drop=True)
    
    df_models_ALL = df_region.loc[df_region["sim"] == "ALL"].copy()
    df_models_HOT = df_region.loc[df_region["sim"] == "HOT"].copy()

    df_models_ALL["quantile"] = pd.qcut(df_models_ALL[target_col], q=100, labels=np.arange(1, 101, 1))
    df_models_HOT["quantile"] = pd.qcut(df_models_HOT[target_col], q=100, labels=np.arange(1, 101, 1))
    
    for values in df_models_ALL.values:
        region_name = values[2]
        model_name = values[1]
        quantile = values[4]

        if region_name not in regional_model_quantiles_ALL.keys():
            regional_model_quantiles_ALL[region_name] = {}
            regional_model_quantiles["Percentile 05 - ALL"][region_name] = []
            regional_model_quantiles["Percentile 10 - ALL"][region_name] = []
            regional_model_quantiles["Percentile 50 - ALL"][region_name] = []
            regional_model_quantiles["Percentile 90 - ALL"][region_name] = []
            regional_model_quantiles["Percentile 95 - ALL"][region_name] = []

        if quantile == 5:
            regional_model_quantiles["Percentile 05 - ALL"][region_name].append(model_name)
        elif quantile == 10:
            regional_model_quantiles["Percentile 10 - ALL"][region_name].append(model_name)
        elif quantile == 50:
            regional_model_quantiles["Percentile 50 - ALL"][region_name].append(model_name)
        elif quantile == 90:
            regional_model_quantiles["Percentile 90 - ALL"][region_name].append(model_name)
        elif quantile == 95:
            regional_model_quantiles["Percentile 95 - ALL"][region_name].append(model_name)

        regional_model_quantiles_ALL[region_name][model_name] = quantile
    
    for values in df_models_HOT.values:
        region_name = values[2]
        model_name = values[1]
        quantile = values[4]

        if region_name not in regional_model_quantiles_HOT.keys():
            regional_model_quantiles_HOT[region_name] = {}
            regional_model_quantiles["Percentile 05 - HOT"][region_name] = []
            regional_model_quantiles["Percentile 10 - HOT"][region_name] = []
            regional_model_quantiles["Percentile 50 - HOT"][region_name] = []
            regional_model_quantiles["Percentile 90 - HOT"][region_name] = []
            regional_model_quantiles["Percentile 95 - HOT"][region_name] = []

        if quantile == 5:
            regional_model_quantiles["Percentile 05 - HOT"][region_name].append(model_name)
        elif quantile == 10:
            regional_model_quantiles["Percentile 10 - HOT"][region_name].append(model_name)
        elif quantile == 50:
            regional_model_quantiles["Percentile 50 - HOT"][region_name].append(model_name)
        elif quantile == 90:
            regional_model_quantiles["Percentile 90 - HOT"][region_name].append(model_name)
        elif quantile == 95:
            regional_model_quantiles["Percentile 95 - HOT"][region_name].append(model_name)

        regional_model_quantiles_HOT[region_name][model_name] = quantile

In [ ]:
write_json_to_disk(regional_model_quantiles_ALL, DIR_HAPPI, "regional_model_quantiles_ALL")
write_json_to_disk(regional_model_quantiles_HOT, DIR_HAPPI, "regional_model_quantiles_HOT")

In [ ]:
summary_dfs = []

for source in tqdm(["ALL", "HOT"], desc="Source Loop", leave=1):
    for region_name in tqdm(df["region"].unique(), desc="Region Loop", leave=1):
        df_region = df.loc[
            (df["region"] == region_name) &
            (df["sim"] == source) 
        ].copy()

        df_region["model_tag"] = df_region["model_tag"].astype(str)

        dfs = []

        for classification in tqdm([
            f"Percentile 05 - {source}",
            f"Percentile 10 - {source}",
            f"Percentile 50 - {source}",
            f"Percentile 90 - {source}",
            f"Percentile 95 - {source}"
        ], desc="Percentile Loop", leave=0):

            df_percentile = df_region.loc[df_region["model_tag"].isin(regional_model_quantiles[classification][region_name])].copy()


            for model_tag in df_percentile["model_tag"].unique():
                df_model = df_percentile.loc[df_percentile["model_tag"] == model_tag].copy()

                df_model["day"] = df_model["time1"].str[8:10]
                group_cols = ["model_tag", "month", "day"]
                df_model = df_model[group_cols + [target_col]].groupby(group_cols).median().reset_index(drop=False)

                df_model.sort_values(by=["model_tag", target_col], ascending=False, inplace=True)
                df_model = df_model.reset_index(drop=True).reset_index(drop=False)

                df_model["cumulative_precip"] = df_model[target_col].cumsum()
                
                # ====================================================================
                # This can be done mulitple ways we should discuss this
                # df_model["percent_of_annual_rainfall"] = df_model["cumulative_precip"] / expected_precip_map_region[region_name]["ALL"] # Expected current decade region total 
                df_model["percent_of_annual_rainfall"] = df_model["cumulative_precip"] / climatology_precip_median # Entire country and climatology total
                
                df_model.rename(columns={"index": "precip_rank"}, inplace=True)
                df_model["classification"] = classification
                dfs.append(df_model)

        df_temp = pd.concat(dfs)

        group_cols = ["precip_rank", "classification"]
        df_temp = df_temp[group_cols + ["cumulative_precip", "percent_of_annual_rainfall"]].groupby(group_cols).median().reset_index(drop=False)
        
        # Add in metadata
        df_temp["region"] = region_name
        df_temp["source"] = source
        
        summary_dfs.append(df_temp)
        
df_summary = pd.concat(summary_dfs)

In [ ]:
df_summary

In [ ]:
# def total_year_determine_model_quantiles(df: pd.DataFrame, target_col: str):
#     # Step: Sort column(s) total_precip_wet_days ascending (A-Z)
#     df = df.sort_values(by=[target_col], ascending=[True])

#     # Step: Split into ALL and HOT
#     df_ALL = df.loc[df["source"] == "ALL"]
#     df_HOT = df.loc[df["source"] == "HOT"]
    
#     # Step: Group them by ensemble member
#     group_cols = [
#         "model"
#     ]
#     df_ALL = df[]
    

#     # Step: Bin these based on wet precip column
#     df_ALL["quantile"] = pd.qcut(df_ALL[target_col], q=100, labels=np.arange(1, 101, 1))
#     df_HOT["quantile"] = pd.qcut(df_HOT[target_col], q=100, labels=np.arange(1, 101, 1))
    
#     # Step: Group them by quantile
#     cols = ["quantile", target_col]
#     df_ALL = df_ALL[cols].groupby(by=["quantile"]).median().reset_index(drop=False)
#     df_HOT = df_HOT[cols].groupby(by=["quantile"]).median().reset_index(drop=False)
    
#     # Step: Merge them into one
#     df_merged = df_HOT.merge(
#         right=df_ALL,
#         on=["quantile"],
#         suffixes=["_HOT", "_ALL"]
#     )
    
#     # Step: Determine differences
#     df_merged["difference"] = df_merged[f"{target_col}_HOT"] / df_merged[f"{target_col}_ALL"]
    
#     return df_merged



def total_year_plot_quantile_difference(df:pd.DataFrame, target_col: str, region_name: str):
    colour_map = {
        "ALL": "rgba(210, 40, 210, 0.9)",
        "HOT": "rgba(210, 10, 10, 0.9)",
        "OBS": "rgba(50, 50, 200, 0.9)",
    }
    
    width_map = {
        "05": 2,
        "10": 2,
        "50": 4,
        "90": 2,
        "95": 2,
    }
    
    fig = go.Figure()
    
    for source in df["source"].unique():
        df_source = df.loc[df["source"] == source]
        for classification in df_source["classification"].unique():
            quantile = re.findall(r"Percentile (.*?) -", classification)[0]
            df_temp = df_source.loc[df_source["classification"] == classification]
            fig.add_trace(
                go.Line( 
                    x=df_temp["precip_rank"],
                    y=df_temp[target_col],
                    line=dict(color=colour_map[source], width=width_map[quantile]),
                    name=classification
                )
            )
    
    fig.update_layout(
        title=f"Region: {region_name} Summary plot",
        height=600, 
        width=900,
        xaxis_range=[0, 30],
        yaxis_range=[0, 0.8]
    )
    
    # fig.update_traces(mode="lines", line_color=colour_map["HOT"])
    
    fig.update_yaxes(title="Cumulative fraction of annual rainfall <br> (with \"annual\" fixed as climatology)")
    fig.update_xaxes(title="Number of days of rain required")
    
    return fig                         

In [ ]:
region_name = "Auckland Region"

df_sample = df_summary.loc[
    (df_summary["region"] == region_name) 
    # (df_summary["source"] == "HOT")
]

total_year_plot_quantile_difference(df_sample, "percent_of_annual_rainfall", region_name)

### Determine model rankings (quantiles) - whole country

In [ ]:
country_model_quantiles_ALL = {}
country_model_quantiles_HOT = {}

country_model_quantiles = {
    # ALL
    "Percentile 05 - ALL": [],
    "Percentile 10 - ALL": [],
    "Percentile 50 - ALL": [],
    "Percentile 90 - ALL": [],
    "Percentile 95 - ALL": [],
    # HOT
    "Percentile 05 - HOT": [],
    "Percentile 10 - HOT": [],
    "Percentile 50 - HOT": [],
    "Percentile 90 - HOT": [],
    "Percentile 95 - HOT": [],
}

group_cols = [
    "sim",
    "model_tag"
]
df_models = df_grouped[group_cols + [target_col]].groupby(by=group_cols).mean().reset_index()
df_models = df_models.sort_values(by=[target_col]).reset_index(drop=True)

df_models_ALL = df_models.loc[df_models["sim"] == "ALL"].copy()
df_models_HOT = df_models.loc[df_models["sim"] == "HOT"].copy()

df_models_ALL["quantile"] = pd.qcut(df_models_ALL[target_col], q=100, labels=np.arange(1, 101, 1))
df_models_HOT["quantile"] = pd.qcut(df_models_HOT[target_col], q=100, labels=np.arange(1, 101, 1))

for values in df_models_ALL.values:
    model_name = values[1]
    quantile = values[3]

    if quantile == 5:
        country_model_quantiles["Percentile 05 - ALL"].append(model_name)
    elif quantile == 10:
        country_model_quantiles["Percentile 10 - ALL"].append(model_name)
    elif quantile == 50:
        country_model_quantiles["Percentile 50 - ALL"].append(model_name)
    elif quantile == 90:
        country_model_quantiles["Percentile 90 - ALL"].append(model_name)
    elif quantile == 95:
        country_model_quantiles["Percentile 95 - ALL"].append(model_name)

    country_model_quantiles_ALL[model_name] = quantile

for values in df_models_HOT.values:
    model_name = values[1]
    quantile = values[3]

    if quantile == 5:
        country_model_quantiles["Percentile 05 - HOT"].append(model_name)
    elif quantile == 10:
        country_model_quantiles["Percentile 10 - HOT"].append(model_name)
    elif quantile == 50:
        country_model_quantiles["Percentile 50 - HOT"].append(model_name)
    elif quantile == 90:
        country_model_quantiles["Percentile 90 - HOT"].append(model_name)
    elif quantile == 95:
        country_model_quantiles["Percentile 95 - HOT"].append(model_name)

    country_model_quantiles_HOT[model_name] = quantile

    
df_models_ALL["quantile"] = df_models_ALL["quantile"].astype(str)
df_models_HOT["quantile"] = df_models_HOT["quantile"].astype(str)

In [ ]:
write_json_to_disk(country_model_quantiles_ALL, DIR_HAPPI, "country_model_quantiles_ALL")
write_json_to_disk(country_model_quantiles_HOT, DIR_HAPPI, "country_model_quantiles_HOT")

In [ ]:
target_quantile = "50"

group_cols = [
    "grid_cell",
]

target_col = "precip_mm"

target_models_ALL = df_models_ALL.loc[df_models_ALL["quantile"] == target_quantile]["model_tag"].values
target_models_HOT = df_models_HOT.loc[df_models_HOT["quantile"] == target_quantile]["model_tag"].values

df_ALL_grouped = df_grouped.loc[df_grouped["model_tag"].isin(target_models_ALL)][group_cols + [target_col]].groupby(by=group_cols).median().reset_index()
df_HOT_grouped = df_grouped.loc[df_grouped["model_tag"].isin(target_models_HOT)][group_cols + [target_col]].groupby(by=group_cols).median().reset_index()

df_merged = df_HOT_grouped.merge(
    right=df_ALL_grouped,
    how="left",
    on="grid_cell",
    suffixes=["_HOT", "_ALL"]
)

df_merged["precip_mm_diff"] = df_merged["precip_mm_HOT"] / df_merged["precip_mm_ALL"]
df_merged["precip_mm_diff"] = df_merged["precip_mm_HOT"] / df_merged["precip_mm_ALL"]

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_merged["grid_cell"].str.split('_', n=2, expand=True)
split_df.columns = ["latitude", "longitude"]
df_merged = pd.merge(df_merged, split_df, how="left", left_index=True, right_index=True)
df_merged["longitude"] = df_merged["longitude"].astype(float)
df_merged["latitude"] = df_merged["latitude"].astype(float)

df_merged

In [ ]:
def determine_country_difference_for_quantile(df_grouped: pd.DataFrame, df_models_ALL: pd.DataFrame, df_models_HOT: pd.DataFrame, target_quantile: str):
    group_cols = ["grid_cell"]
    target_col = "precip_mm"

    target_models_ALL = df_models_ALL.loc[df_models_ALL["quantile"] == target_quantile]["model_tag"].values
    target_models_HOT = df_models_HOT.loc[df_models_HOT["quantile"] == target_quantile]["model_tag"].values

    df_ALL_grouped = df_grouped.loc[df_grouped["model_tag"].isin(target_models_ALL)][group_cols + [target_col]].groupby(by=group_cols).median().reset_index()
    df_HOT_grouped = df_grouped.loc[df_grouped["model_tag"].isin(target_models_HOT)][group_cols + [target_col]].groupby(by=group_cols).median().reset_index()

    df_merged = df_HOT_grouped.merge(
        right=df_ALL_grouped,
        how="left",
        on="grid_cell",
        suffixes=["_HOT", "_ALL"]
    )

    df_merged["precip_mm_diff"] = df_merged["precip_mm_HOT"] / df_merged["precip_mm_ALL"]
    df_merged["precip_mm_diff"] = df_merged["precip_mm_HOT"] / df_merged["precip_mm_ALL"]

    # Step: Manipulate strings of 'grid_cell' and perform a split on '_'
    split_df = df_merged["grid_cell"].str.split('_', n=2, expand=True)
    split_df.columns = ["latitude", "longitude"]
    df_merged = pd.merge(df_merged, split_df, how="left", left_index=True, right_index=True)
    df_merged["longitude"] = df_merged["longitude"].astype(float)
    df_merged["latitude"] = df_merged["latitude"].astype(float)

    return df_merged



def plot_country_difference_map(df_merged, quantile):
    if int(quantile) < 50:
        chosen_colour_map = "OrRd"
    elif int(quantile) == 50:
        chosen_colour_map = "RdBu"
    else:
        chosen_colour_map = "GnBu"
    
    
    fig = px.scatter(
        df_merged, 
        x='longitude', 
        y='latitude', 
        color='precip_mm_diff', 
        symbol_sequence=['circle'], 
        height=800, 
        width=800,
        template="plotly_white",
        color_continuous_scale=chosen_colour_map,
        range_color=(0.8, 1.2)
    )
    
    fig.update_layout(title=f"Fraction Precipitation Difference (HOT / ALL) for the quantile {quantile} ensemble members")

    fig.update_traces(marker={'size': 21})

    return fig



def plot_country_difference_map_geo(df_merged, quantile):
    if int(quantile) < 50:
        chosen_colour_map = "OrRd"
    elif int(quantile) == 50:
        chosen_colour_map = "RdBu"
    else:
        chosen_colour_map = "GnBu"
    
    fig = go.Figure()
    
    fig.add_trace(
        go.Scattergeo( 
            lon=df_merged['longitude'], 
            lat=df_merged['latitude'], 
            # color=df_merged['precip_mm_diff'], 
            symbol_sequence=['circle'], 
            height=800, 
            width=800,
            template="plotly_white",
            color_continuous_scale=chosen_colour_map,
            range_color=(0.8, 1.2)
        )
    )
    
    fig.update_layout(title=f"Fraction Precipitation Difference (HOT / ALL) for the quantile {quantile} ensemble members")

    fig.update_traces(marker={'size': 21})

    return fig


In [ ]:
target_quantile = "90"
df_merged = determine_country_difference_for_quantile(df_grouped, df_models_ALL, df_models_HOT, target_quantile)
plot_country_difference_map(df_merged, target_quantile)

In [ ]:
target_quantile = "10"
df_merged = determine_country_difference_for_quantile(df_grouped, df_models_ALL, df_models_HOT, target_quantile)
plot_country_difference_map(df_merged, target_quantile)

# Part 5 - Sensitivity tests

## RxDay

In [ ]:
def determine_quantiles_for_rx_data(df_ALL, df_HOT, num_quantiles=100):
    dfs = []

    # Iterate over each grid cell
    for grid_cell in tqdm(df_ALL["grid_cell"].unique(), desc="Grid_cell Loop"):
        # Filter to that grid cell data
        df_temp_ALL = df_ALL.loc[df_ALL["grid_cell"] == grid_cell].copy()
        df_temp_HOT = df_HOT.loc[df_HOT["grid_cell"] == grid_cell].copy()

        # Determine wet quantiles
        df_temp_ALL["wet_quantile"] = pd.qcut(df_temp_ALL["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_HOT["wet_quantile"] = pd.qcut(df_temp_HOT["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        
        # Change the quantile cols to strings then integers as they start as objects
        df_temp_ALL["wet_quantile"] = df_temp_ALL["wet_quantile"].astype(str).astype(int)
        df_temp_HOT["wet_quantile"] = df_temp_HOT["wet_quantile"].astype(str).astype(int)
        
        # ==================================================================================================================================
        # I am quite tired and seem to have broken the "merge" function. I can't be bothered fixing it so this is a fast workaround

        # Group by the quantiles taking the median of each quantile, this "bins them"
        group_cols = [
            "grid_cell",
        ]
        wet_cols = [
            "total_precip_wet_days",
            "wet_quantile",
        ]
        df_ALL_wet = df_temp_ALL[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        df_HOT_wet = df_temp_HOT[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        
        # Merge them back into one
        df_merged = df_ALL_wet[group_cols]
        for col in [
            "total_precip_wet_days",
            "wet_quantile",
        ]:
            df_merged[f"{col}_HOT"] = df_HOT_wet[col]
            df_merged[f"{col}_ALL"] = df_ALL_wet[col]
        
        # ==================================================================================================================================

        # Determine total_precip_xxx_days_fraction here to save time
        df_merged["total_precip_wet_days_fraction"] = (df_merged["total_precip_wet_days_HOT"] / df_merged["total_precip_wet_days_ALL"]).astype(float).round(3)
        
        # Pull out the lon and lat values
        grid_cell_parts = grid_cell.split("_")
        lat = float(grid_cell_parts[0])
        lon = float(grid_cell_parts[1])
        df_merged["longitude"] = lon
        df_merged["latitude"] = lat

        # Append this to the big list
        dfs.append(df_merged)

    # Concat all the dfs into one giga-one
    return pd.concat(dfs)


def plot_difference_map_rx_quantile(df_merged, target_quantile, target_scenario, rx):
    colour_map = {
        "dry_days": "OrRd",
        "wet_days": "GnBu"
    }
    
    fig = px.scatter(
        df_merged, 
        x='longitude', 
        y='latitude', 
        color=f"total_precip_{target_scenario}_fraction", 
        symbol_sequence=["circle"], 
        height=900, 
        width=900,
        template="plotly_white",
        color_continuous_scale=colour_map[target_scenario],
        range_color=(0.80, 1.25)
    )
    
    fig.update_layout(
        title=f"Rx{rx}Day Fraction Precipitation Difference (HOT / ALL) {target_scenario.replace('_', ' ')} [ {target_quantile}th quantile ]",
        yaxis_title=None,
        xaxis_title=None
    )
    
    # fig.update_xaxes(visible=False)
    # fig.update_yaxes(visible=False)
    
    fig.update_traces(marker={'size': 21})

    return fig

In [ ]:
sel = [("region", "!=", "Area Outside Region")]

df = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_RX/", filters=sel)

In [ ]:
df_ALL = df.loc[df["source"] == "ALL"].copy().reset_index(drop=True)
df_HOT = df.loc[df["source"] == "HOT"].copy().reset_index(drop=True)

In [ ]:
df_merged_1 = determine_quantiles_for_rx_data(df_ALL.loc[df_ALL["window"] == 1].copy(), df_HOT.loc[df_HOT["window"] == 1].copy())
df_merged_3 = determine_quantiles_for_rx_data(df_ALL.loc[df_ALL["window"] == 3].copy(), df_HOT.loc[df_HOT["window"] == 3].copy())
df_merged_5 = determine_quantiles_for_rx_data(df_ALL.loc[df_ALL["window"] == 5].copy(), df_HOT.loc[df_HOT["window"] == 5].copy())
df_merged_7 = determine_quantiles_for_rx_data(df_ALL.loc[df_ALL["window"] == 7].copy(), df_HOT.loc[df_HOT["window"] == 7].copy())

In [ ]:
target_quantile = 10

df_quantile_1 = df_merged_1.loc[df_merged_1["wet_quantile_ALL"] == target_quantile].copy()
df_quantile_3 = df_merged_3.loc[df_merged_1["wet_quantile_ALL"] == target_quantile].copy()
df_quantile_5 = df_merged_5.loc[df_merged_1["wet_quantile_ALL"] == target_quantile].copy()
df_quantile_7 = df_merged_7.loc[df_merged_1["wet_quantile_ALL"] == target_quantile].copy()

fig_wet_1 = plot_difference_map_rx_quantile(df_quantile_1, target_quantile, "wet_days", 1)
fig_wet_3 = plot_difference_map_rx_quantile(df_quantile_3, target_quantile, "wet_days", 3)
fig_wet_5 = plot_difference_map_rx_quantile(df_quantile_5, target_quantile, "wet_days", 5)
fig_wet_7 = plot_difference_map_rx_quantile(df_quantile_7, target_quantile, "wet_days", 7)

fig_wet_1.show()
fig_wet_3.show()
fig_wet_5.show()
fig_wet_7.show()

## Constant fixed 5 wet 300 dry days

In [ ]:
def determine_quantiles_for_data(df_ALL, df_HOT, num_quantiles=100):
    dfs = []

    # Iterate over each grid cell
    for grid_cell in tqdm(df_ALL["grid_cell"].unique(), desc="Grid_cell Loop"):
        # Filter to that grid cell data
        df_temp_ALL = df_ALL.loc[df_ALL["grid_cell"] == grid_cell].copy()
        df_temp_HOT = df_HOT.loc[df_HOT["grid_cell"] == grid_cell].copy()

        # Determine wet and dry quantiles
        df_temp_ALL["wet_quantile"] = pd.qcut(df_temp_ALL["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_ALL["dry_quantile"] = pd.qcut(df_temp_ALL["total_precip_dry_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_HOT["wet_quantile"] = pd.qcut(df_temp_HOT["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_HOT["dry_quantile"] = pd.qcut(df_temp_HOT["total_precip_dry_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))

        # Change the quantile cols to strings then integers as they start as objects
        df_temp_ALL["wet_quantile"] = df_temp_ALL["wet_quantile"].astype(str).astype(int)
        df_temp_ALL["dry_quantile"] = df_temp_ALL["dry_quantile"].astype(str).astype(int)
        df_temp_HOT["wet_quantile"] = df_temp_HOT["wet_quantile"].astype(str).astype(int)
        df_temp_HOT["dry_quantile"] = df_temp_HOT["dry_quantile"].astype(str).astype(int)

        # ==================================================================================================================================
        # I am quite tired and seem to have broken the "merge" function. I can't be bothered fixing it so this is a fast workaround

        # Group by the quantiles taking the median of each quantile, this "bins them"
        group_cols = [
            "grid_cell",
            "days_wet",
            "days_dry",
        ]
        wet_cols = [
            "total_precip_wet_days",
            "wet_quantile",
        ]
        dry_cols = [
            "total_precip_dry_days",
            "dry_quantile"
        ]
        df_ALL_wet = df_temp_ALL[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        df_ALL_dry = df_temp_ALL[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
        df_ALL_wet["total_precip_dry_days"] = df_ALL_dry["total_precip_dry_days"]
        df_ALL_wet["dry_quantile"] = df_ALL_dry["dry_quantile"]
        df_HOT_wet = df_temp_HOT[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        df_HOT_dry = df_temp_HOT[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
        df_HOT_wet["total_precip_dry_days"] = df_HOT_dry["total_precip_dry_days"]
        df_HOT_wet["dry_quantile"] = df_HOT_dry["dry_quantile"]

        # Merge them back into one
        df_merged = df_ALL_wet[group_cols]
        for col in [
            "total_precip_wet_days",
            "total_precip_dry_days",
            "wet_quantile",
            "dry_quantile",
        ]:
            if "wet" in col:
                df_merged[f"{col}_HOT"] = df_HOT_wet[col]
                df_merged[f"{col}_ALL"] = df_ALL_wet[col]
            else:
                df_merged[f"{col}_HOT"] = df_HOT_dry[col]
                df_merged[f"{col}_ALL"] = df_ALL_dry[col]

        # ==================================================================================================================================

        # Determine total_precip_xxx_days_fraction here to save time
        df_merged["total_precip_wet_days_fraction"] = (df_merged["total_precip_wet_days_HOT"] / df_merged["total_precip_wet_days_ALL"]).astype(float).round(3)
        df_merged["total_precip_dry_days_fraction"] = (df_merged["total_precip_dry_days_HOT"] / df_merged["total_precip_dry_days_ALL"]).astype(float).round(3)
        
        
        # Pull out the lon and lat values
        grid_cell_parts = grid_cell.split("_")
        lat = float(grid_cell_parts[0])
        lon = float(grid_cell_parts[1])
        df_merged["longitude"] = lon
        df_merged["latitude"] = lat

        # Append this to the big list
        dfs.append(df_merged)

    # Concat all the dfs into one giga-one
    return pd.concat(dfs)
    


def plot_difference_map_quantile(df_merged, target_quantile, target_scenario):
    colour_map = {
        "dry_days": "OrRd",
        "wet_days": "GnBu"
    }
    
    fig = px.scatter(
        df_merged, 
        x='longitude', 
        y='latitude', 
        color=f"total_precip_{target_scenario}_fraction", 
        symbol_sequence=["circle"], 
        height=900, 
        width=900,
        template="plotly_white",
        color_continuous_scale=colour_map[target_scenario],
        range_color=(0.80, 1.25)
    )
    
    fig.update_layout(
        title=f"Fraction Precipitation Difference (HOT / ALL) {target_scenario.replace('_', ' ')} [ {target_quantile}th quantile fixed 5 wet 300 dry days ]",
        yaxis_title=None,
        xaxis_title=None
    )
    
    # fig.update_xaxes(visible=False)
    # fig.update_yaxes(visible=False)
    
    fig.update_traces(marker={'size': 21})

    return fig

In [ ]:
sel = [("region", "!=", "Area Outside Region")]

df = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_Days_5_300/", filters=sel)

df_ALL = df.loc[df["source"] == "ALL"].copy().reset_index(drop=True)
df_HOT = df.loc[df["source"] == "HOT"].copy().reset_index(drop=True)

df_ALL["days_wet"] = 5
df_ALL["days_dry"] = 300
df_HOT["days_wet"] = 5
df_HOT["days_dry"] = 300

In [ ]:
df_merged = determine_quantiles_for_data(df_ALL, df_HOT, num_quantiles=100)

In [ ]:
df_merged.to_parquet(f"{DIR_HAPPI}/RVI/fixed_day_5_300_grid_cell_quantiles.parquet")

df_merged.to_csv(f"{DIR_HAPPI}/RVI/fixed_day_5_300_grid_cell_quantiles.csv")

In [ ]:
target_quantile = 50
df_quantile = df_merged.loc[df_merged["dry_quantile_ALL"] == target_quantile].copy()

fig_wet = plot_difference_map_quantile(df_quantile, target_quantile, "wet_days")
fig_dry = plot_difference_map_quantile(df_quantile, target_quantile, "dry_days")

fig_wet.show()
fig_dry.show()

fig_wet.write_html(f"{DIR_PLOTS}/NZ_ratio_5_300_{target_quantile}th_quantile_wet.html")
fig_dry.write_html(f"{DIR_PLOTS}/NZ_ratio_5_300_{target_quantile}th_quantile_dry.html")

fig_wet.write_image(f"{DIR_PLOTS}/NZ_ratio_5_300_{target_quantile}th_quantile_wet.png", format="png", scale=2)
fig_dry.write_image(f"{DIR_PLOTS}/NZ_ratio_5_300_{target_quantile}th_quantile_dry.png", format="png", scale=2)

# Part X - Supporting Plots for Thesis

In [ ]:
sel = [("region", "!=", "Area Outside Region")]

df = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_Days/", filters=sel)

target_regions = ["Northland Region", "Tasman Region", "Southland Region"]

print(df["region"].value_counts(normalize=True))

# df = df.loc[df["region"].isin(target_regions)]

In [ ]:
grid_cells_fiordland = [
    "-44.22428_167.67783",
    "-44.74854_167.24373",
    "-44.652485_167.81955",
    "-45.271618_166.80473",
    "-45.177055_167.38486",
    "-45.08052_167.96336",
    "-45.793488_166.36075",
    "-45.70044_166.9453",
    "-45.605396_167.52815",
    "-46.222622_166.50067",
    "-46.129093_167.08798",
    "-46.033554_167.67365",
    "-46.65158_166.64278",
    "-46.557556_167.23296",
    "-46.46152_167.82141",
]

df.loc[df["grid_cell"].isin(grid_cells_fiordland), "region"] = "Fiordland Region"
df.loc[df["grid_cell"].isin(grid_cells_fiordland), "region"] = "Fiordland Region"

In [ ]:
def determine_quantiles_for_data(df_ALL, df_HOT, num_quantiles=100):
    dfs = []

    # Iterate over each grid cell
    for grid_cell in tqdm(df_ALL["grid_cell"].unique(), desc="Grid_cell Loop"):
        # Filter to that grid cell data
        df_temp_ALL = df_ALL.loc[df_ALL["grid_cell"] == grid_cell].copy()
        df_temp_HOT = df_HOT.loc[df_HOT["grid_cell"] == grid_cell].copy()

        # Determine wet and dry quantiles
        df_temp_ALL["wet_quantile"] = pd.qcut(df_temp_ALL["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_ALL["dry_quantile"] = pd.qcut(df_temp_ALL["total_precip_dry_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_HOT["wet_quantile"] = pd.qcut(df_temp_HOT["total_precip_wet_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))
        df_temp_HOT["dry_quantile"] = pd.qcut(df_temp_HOT["total_precip_dry_days"], q=num_quantiles, labels=np.arange(1, num_quantiles + 1, 1))

        # Change the quantile cols to strings then integers as they start as objects
        df_temp_ALL["wet_quantile"] = df_temp_ALL["wet_quantile"].astype(str).astype(int)
        df_temp_ALL["dry_quantile"] = df_temp_ALL["dry_quantile"].astype(str).astype(int)
        df_temp_HOT["wet_quantile"] = df_temp_HOT["wet_quantile"].astype(str).astype(int)
        df_temp_HOT["dry_quantile"] = df_temp_HOT["dry_quantile"].astype(str).astype(int)

        # ==================================================================================================================================
        # I am quite tired and seem to have broken the "merge" function. I can't be bothered fixing it so this is a fast workaround

        # Group by the quantiles taking the median of each quantile, this "bins them"
        group_cols = [
            "grid_cell",
            "days_wet",
            "days_dry",
        ]
        wet_cols = [
            "total_precip_wet_days",
            "wet_quantile",
        ]
        dry_cols = [
            "total_precip_dry_days",
            "dry_quantile"
        ]
        df_ALL_wet = df_temp_ALL[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        df_ALL_dry = df_temp_ALL[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
        df_ALL_wet["total_precip_dry_days"] = df_ALL_dry["total_precip_dry_days"]
        df_ALL_wet["dry_quantile"] = df_ALL_dry["dry_quantile"]
        df_HOT_wet = df_temp_HOT[group_cols + ["wet_quantile", "total_precip_wet_days"]].groupby(by=group_cols + ["wet_quantile"]).median().reset_index().sort_values("wet_quantile", ascending=True)
        df_HOT_dry = df_temp_HOT[group_cols + ["dry_quantile", "total_precip_dry_days"]].groupby(by=group_cols + ["dry_quantile"]).median().reset_index().sort_values("dry_quantile", ascending=True)
        df_HOT_wet["total_precip_dry_days"] = df_HOT_dry["total_precip_dry_days"]
        df_HOT_wet["dry_quantile"] = df_HOT_dry["dry_quantile"]

        # Merge them back into one
        df_merged = df_ALL_wet[group_cols]
        for col in [
            "total_precip_wet_days",
            "total_precip_dry_days",
            "wet_quantile",
            "dry_quantile",
        ]:
            if "wet" in col:
                df_merged[f"{col}_HOT"] = df_HOT_wet[col]
                df_merged[f"{col}_ALL"] = df_ALL_wet[col]
            else:
                df_merged[f"{col}_HOT"] = df_HOT_dry[col]
                df_merged[f"{col}_ALL"] = df_ALL_dry[col]

        # ==================================================================================================================================

        # Determine total_precip_xxx_days_fraction here to save time
        df_merged["total_precip_wet_days_fraction"] = (df_merged["total_precip_wet_days_HOT"] / df_merged["total_precip_wet_days_ALL"]).astype(float).round(3)
        df_merged["total_precip_dry_days_fraction"] = (df_merged["total_precip_dry_days_HOT"] / df_merged["total_precip_dry_days_ALL"]).astype(float).round(3)
        
        
        # Pull out the lon and lat values
        grid_cell_parts = grid_cell.split("_")
        lat = float(grid_cell_parts[0])
        lon = float(grid_cell_parts[1])
        df_merged["longitude"] = lon
        df_merged["latitude"] = lat

        # Append this to the big list
        dfs.append(df_merged)

    # Concat all the dfs into one giga-one
    return pd.concat(dfs)

In [ ]:
df_ALL = df.loc[df["source"] == "ALL"].copy().reset_index(drop=True)
df_HOT = df.loc[df["source"] == "HOT"].copy().reset_index(drop=True)


# Attach days required to datasets
with open(f"{DIR_HAPPI}/RVI/fixed_precip_grid_cell_map.json") as json_file:
    fixed_precip_grid_cell_map = load(json_file)
    
fixed_precip_grid_cell_map_wet = {}
fixed_precip_grid_cell_map_dry = {}

for key, value in fixed_precip_grid_cell_map.items():
    fixed_precip_grid_cell_map_wet[key] = int(value["wet"])
    fixed_precip_grid_cell_map_dry[key] = int(value["dry"])

df_ALL["days_wet"] = df_ALL["grid_cell"].map(fixed_precip_grid_cell_map_wet)
df_ALL["days_dry"] = df_ALL["grid_cell"].map(fixed_precip_grid_cell_map_dry)
df_HOT["days_wet"] = df_HOT["grid_cell"].map(fixed_precip_grid_cell_map_wet)
df_HOT["days_dry"] = df_HOT["grid_cell"].map(fixed_precip_grid_cell_map_dry)

## The line plots comparing ensemble members against each other in the grid_cell (HOT / ALL for wet and dry)

In [ ]:
region_figures = dict()

df_merged = pd.DataFrame()

for region in target_regions + ["Fiordland Region"]:
    fig = go.Figure()
    
    df_region_ALL = df_ALL.loc[df_ALL["region"] == region]
    df_region_HOT = df_HOT.loc[df_HOT["region"] == region]
    
    df_region_merged = determine_quantiles_for_data(df_region_ALL, df_region_HOT, 100)
    df_region_merged.rename(
        columns={
            "total_precip_wet_days_fraction": "Wet End", 
            "total_precip_dry_days_fraction": "Dry End",
            "wet_quantile_ALL": "Ranked Bin"
        }, 
        inplace=True
    )
    
    df_region_merged["region"] = region
    
    df_merged = pd.concat([df_merged, df_region_merged]).reset_index(drop=True)
    
    fig = px.scatter(
        df_region_merged,
        x="Ranked Bin",
        y=["Wet End", "Dry End"],
        template="plotly_white",
        color_discrete_sequence=px.colors.qualitative.Bold[:]
    )
    
    fig.update_layout(
        title=f"Comparing Expected Precipitation Volume HOT / ALL | {region}",
        height=600,
        width=1000,
        yaxis_range=[0.75, 1.25],
        legend_title_text="",
        legend=dict(
            orientation="h",
            yanchor="top",
            y=0.95,
            xanchor="left",
            x=0.05
        )
    )
    
    fig.update_xaxes(title="Ranked Bin (splitting each quartile into 100 further bins)")
    fig.update_yaxes(title="Change in Expected Precipitation (relative scale)")
    
    fig.show()
    
    fig.write_image(f"../Plots_Thesis/LineComparison_{region.replace(' ', '')}.png", scale=2)
    
    region_figures[region] = fig

## The histograms

In [ ]:
import math

from plotly.subplots import make_subplots


for region in target_regions + ["Fiordland Region"]:
    
    df_region = df.loc[df["region"] == region].copy()
    grid_cells = df_region["grid_cell"].unique()
    num_grid_cells = len(grid_cells)
    print(f"For {region} - found {num_grid_cells} grid cells")
    
    for theme in ["dry", "wet"]:
    
        fig = make_subplots(
            cols=3, 
            rows=math.ceil(num_grid_cells / 3),
            subplot_titles=tuple(grid_cells)
        )
        row = 1
        col = 1
        
        for grid_cell in grid_cells:
            
            df_cell = df_region.loc[df_region["grid_cell"] == grid_cell]
            
            values_ALL = df_cell.loc[df_cell["source"] == "ALL"][f"total_precip_{theme}_days"]
            values_HOT = df_cell.loc[df_cell["source"] == "HOT"][f"total_precip_{theme}_days"]
            opacity = 0.6
            num_bins = 150
            
            fig.add_trace(
                go.Histogram(
                    x=values_ALL, 
                    # xbins={"start": 0, "end": values_ALL.max(), "size":10},
                    nbinsx=num_bins,
                    opacity=opacity,
                    marker_color="blue",
                    name="ALL"
                ),
                col=col,
                row=row
            )
            
            fig.add_trace(
                go.Histogram(
                    x=values_HOT, 
                    # xbins={"start": 0, "end": values_HOT.max(), "size":10},
                    nbinsx=num_bins,
                    opacity=opacity,
                    marker_color="red",
                    name="HOT"
                ),
                col=col,
                row=row
            )
            
            col += 1
            if col == 4:
                col = 1
                row += 1

        fig.update_traces(marker_line_width=0.5,marker_line_color="black")
        
        if region == "Tasman Region":
            height = num_grid_cells * 120
        else:
            height = num_grid_cells * 80
        
        fig.update_layout(
            title=f"{region} - Ensemble members 25% annual precipitation volume histograms for {theme} end",
            height=height,
            width=1000,
            showlegend=False,
            barmode="overlay",
            template="plotly_white"
        )

        fig.update_yaxes(title_text="")
        fig.update_xaxes(title_text="")

        fig.show()

        fig.write_image(f"../Plots_Thesis/Histograms_{region.replace(' ', '')}_{theme}.png", scale=2)
        
    # break


## Determine stats for each region

In [ ]:
chosen_region = "Southland Region"

df_temp_ALL = df_ALL.loc[df_ALL["region"] == chosen_region]
df_temp_HOT = df_HOT.loc[df_HOT["region"] == chosen_region]

In [ ]:
df_temp_ALL

In [ ]:
bam.plot(df_temp_ALL, "total_precip_dry_days")

In [ ]:
bam.plot(df_temp_ALL, "total_precip_wet_days")

In [ ]:
bam.plot(df_temp_HOT, "total_precip_dry_days")

In [ ]:
bam.plot(df_temp_HOT, "total_precip_wet_days")

In [ ]:
bam.plot(df_temp_ALL, 'days_wet')

In [ ]:
bam.plot(df_temp_ALL, 'days_dry')

## Single ensemble member rainfall plot

In [ ]:
def add_line(fig, x_val, y_val, colour, name):
    fig.add_shape(
        type='line', 
        x0=x_val, 
        x1=x_val, 
        y0=0, 
        y1=y_val, 
        opacity=0.85,
        line=dict(color=colour, width=3, dash='dash'),
        xref='x', 
        yref='y', 
        layer='above',
        name=name,
        showlegend=True
    )


In [ ]:
df_single = pd.read_parquet(f"{DIR_HAPPI}/Processed/year=2010/region=Northland Region/sim=ALL/model_tag=a0t6/")
df_single = df_single.loc[df_single["grid_cell"] == "-34.648323_172.58727"]
df_single = df_single.sort_values(by=["precip_mm"], ascending=False).reset_index(drop=True)
# df_single

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        y=df_single["precip_mm"], 
        opacity=0.6,
        marker_color="green",
        name="Daily Precipitation"
    )
)

fig.update_traces(marker_line_width=0.5,marker_line_color="black")

fig.update_layout(
    title=f"Single ensemble member precipitation | Grid Cell -34.648323_172.58727 | Model a0t6 | 2010",
    height=600,
    width=1000,
    showlegend=True,
    barmode="overlay",
    template="plotly_white",
    legend=dict(
        yanchor="top",
        y=0.90,
        xanchor="left",
        x=0.75
    )
)

add_line(fig, 5, 70, "blue", "Wet Days Cutoff")
add_line(fig, 42, 70, "red", "Dry Days Cutoff")

fig.update_yaxes(title_text="Daily Precipitation (mm)")
fig.update_xaxes(title_text="Ranked Days")

fig.show()

In [ ]:
fig.write_image(f"../Plots_Thesis/RankedPrecipitation_NorthlandRegion.png", scale=2)

## Country wide change plots

In [ ]:
def plot_difference_map_quantile(df_merged, target_quantile, target_scenario):
    colour_map = {
        "Dry End": "OrRd",
        "Wet End": "GnBu"
    }
    
    fig = px.scatter(
        df_merged, 
        x='longitude', 
        y='latitude', 
        color=f"{target_scenario}", 
        symbol_sequence=["circle"], 
        height=900, 
        width=800,
        template="plotly_white",
        color_continuous_scale=colour_map[target_scenario],
        range_color=(0.75, 1.25)
    )
    
    fig.update_layout(
        title=f"Change in Expected Precipitation (HOT / ALL) {target_scenario} [ {target_quantile}th bin ]",
        coloraxis_colorbar=dict(title="")
    )
    
    fig.update_xaxes(title="Longitude", visible=True)
    fig.update_yaxes(title="Latitude", visible=True)
    
    fig.update_traces(marker={'size': 21})

    return fig

In [ ]:
df_merged = determine_quantiles_for_data(df_ALL, df_HOT, 100)

In [ ]:
df_merged.rename(
    columns={
        "total_precip_wet_days_fraction": "Wet End", 
        "total_precip_dry_days_fraction": "Dry End"
    }, 
    inplace=True
)

In [ ]:
for target_quantile in [10]:
    df_quantile = df_merged.loc[df_merged["dry_quantile_ALL"] == target_quantile].copy()

    fig_wet = plot_difference_map_quantile(df_quantile, target_quantile, "Wet End")
    fig_dry = plot_difference_map_quantile(df_quantile, target_quantile, "Dry End")

    fig_wet.show()
    fig_dry.show()

#     fig_wet.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.html")
#     fig_dry.write_html(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.html")

#     fig_wet.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_wet.png", format="png", scale=2)
#     fig_dry.write_image(f"{DIR_PLOTS}/NZ_change_{target_quantile}th_quantile_dry.png", format="png", scale=2)

In [ ]:
df_quantile = df_quantile.set_geometry(
    geopandas.points_from_xy(
        x=df_quantile["longitude"],
        y=df_quantile["latitude"],
        crs="EPSG:4326"
    )
)

df_quantile.explore("Dry End")

# Review Requests - Days taken to reach 25% for all grid-cells

## For paper

In [ ]:
group_cols = [
    "region",
    "grid_cell",
]

df_wah = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_PrecipPercent")
df_wah = df_wah.loc[df_wah["source"] == "ALL"].reset_index(drop=True)

df_wah_grouped = df_wah[group_cols + ["wet", "dry"]].groupby(group_cols).agg({"wet": "median", "dry": "median"}).reset_index(drop=False)

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_wah_grouped['grid_cell'].str.split('_', n=2, expand=True)
split_df.columns = ["lat", "lon"]
split_df["lat"] = split_df["lat"].astype(float)
split_df["lon"] = split_df["lon"].astype(float)
df_wah_grouped = pd.merge(df_wah_grouped, split_df, how="left", left_index=True, right_index=True)

df_wah_grouped["wet"] = np.floor(df_wah_grouped["wet"])
df_wah_grouped["dry"] = np.floor(df_wah_grouped["dry"])



df_vcsn = pd.read_parquet(f"{DIR_VCSN}/RVI/Chunks_Fixed_PrecipPercent/")
df_vcsn = df_vcsn.loc[df_vcsn["year"].astype(int).between(2007, 2016)]

df_vcsn_grouped = df_vcsn[group_cols + ["wet", "dry"]].groupby(group_cols).agg({"wet": "median", "dry": "median"}).reset_index(drop=False)

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_vcsn_grouped['grid_cell'].str.split('_', n=2, expand=True)
split_df.columns = ["lat", "lon"]
split_df["lat"] = split_df["lat"].astype(float)
split_df["lon"] = split_df["lon"].astype(float)
df_vcsn_grouped = pd.merge(df_vcsn_grouped, split_df, how="left", left_index=True, right_index=True)

df_vcsn_grouped["wet"] = np.floor(df_vcsn_grouped["wet"])
df_vcsn_grouped["dry"] = np.floor(df_vcsn_grouped["dry"])

In [ ]:
colour_map_wet = {
    "wet_0": "rgb(0.478431373, 0.003921569, 0.466666667)",
    "wet_1": "rgb(0.682352941, 0.003921569, 0.494117647)",
    "wet_2": "rgb(0.866666667, 0.203921569, 0.592156863)",
    "wet_3": "rgb(0.968627451, 0.407843137, 0.631372549)",
    "wet_4": "rgb(0.980392157, 0.623529412, 0.709803922)",
    "wet_5": "rgb(0.988235294, 0.772549020, 0.752941176)",
    "wet_6": "rgb(0.996078431, 0.921568627, 0.886274510)",
}

colour_map_dry = {
    "dry_0": "rgb(1.000000000, 1.000000000, 0.831372549)",
    "dry_1": "rgb(0.996078431, 0.890196078, 0.568627451)",
    "dry_2": "rgb(0.996078431, 0.768627451, 0.309803922)",
    "dry_3": "rgb(0.996078431, 0.600000000, 0.160784314)",
    "dry_4": "rgb(0.925490196, 0.439215686, 0.078431373)",
    "dry_5": "rgb(0.800000000, 0.298039216, 0.007843137)",
    "dry_6": "rgb(0.549019608, 0.176470588, 0.015686275)",
}

colour_boundaries = {
    "VCSN" : {
        "wet": [
            (0.000, colour_map_wet["wet_0"]), (0.167, colour_map_wet["wet_0"]),   
            (0.167, colour_map_wet["wet_1"]), (0.333, colour_map_wet["wet_1"]),
            (0.333, colour_map_wet["wet_2"]), (0.500, colour_map_wet["wet_2"]),
            (0.500, colour_map_wet["wet_3"]), (0.667, colour_map_wet["wet_3"]),
            (0.667, colour_map_wet["wet_4"]), (0.833, colour_map_wet["wet_4"]),
            (0.833, colour_map_wet["wet_5"]), (1.000, colour_map_wet["wet_5"]),
            (1.000, colour_map_wet["wet_6"]), (1.000, colour_map_wet["wet_6"]),
        ],
        "dry": [
            (0.000, colour_map_dry["dry_0"]), (0.045, colour_map_dry["dry_0"]),
            (0.045, colour_map_dry["dry_1"]), (0.197, colour_map_dry["dry_1"]),
            (0.197, colour_map_dry["dry_2"]), (0.348, colour_map_dry["dry_2"]),
            (0.348, colour_map_dry["dry_3"]), (0.500, colour_map_dry["dry_3"]),
            (0.500, colour_map_dry["dry_4"]), (0.652, colour_map_dry["dry_4"]),
            (0.652, colour_map_dry["dry_5"]), (0.803, colour_map_dry["dry_5"]),
            (0.803, colour_map_dry["dry_6"]), (1.000, colour_map_dry["dry_6"]),
        ]
    },
    "WaH ALL": {
        "wet": [
            (0.000, colour_map_wet["wet_0"]), (0.100, colour_map_wet["wet_0"]),
            (0.100, colour_map_wet["wet_1"]), (0.200, colour_map_wet["wet_1"]),
            (0.200, colour_map_wet["wet_2"]), (0.300, colour_map_wet["wet_2"]),
            (0.300, colour_map_wet["wet_3"]), (0.400, colour_map_wet["wet_3"]),
            (0.400, colour_map_wet["wet_4"]), (0.500, colour_map_wet["wet_4"]),
            (0.500, colour_map_wet["wet_5"]), (0.600, colour_map_wet["wet_5"]),
            (0.600, colour_map_wet["wet_6"]), (1.000, colour_map_wet["wet_6"]),
        ],
        "dry": [
            
        ]
    }
}

colour_midpoints = {
    "wet": 8,
    "dry": 310
}

tick_vals = {
    "wet": [2, 4, 6, 8, 10, 12, 14, 16],
    "dry": [270, 280, 290, 300, 310, 320, 330, 340, 350],
}
    
tick_text = {
    "wet": ["2", "4", "6", "8", "10", "12", "14", "16+"],
    "dry": ["270", "280", "290", "300", "310", "320", "330", "340+"],
}


In [ ]:
def build_day_plot(df, feature, data_source):
    fig = px.scatter(
        df, 
        x='lon', 
        y='lat', 
        height=950, 
        width=800, 
        color=feature,
        template="plotly_white",
        color_continuous_scale=colour_boundaries[data_source][feature],
        color_continuous_midpoint=colour_midpoints[feature]
    )
    
    amount_str = "Fewest" if feature == "wet" else "Most"
    
    fig.update_layout(
        title=f"{amount_str} days needed to reach 25% of annual rainfall - {data_source}",
        coloraxis_colorbar=dict(
            title="",
            orientation="h",
            y=-0.2,
            yanchor="bottom",
            ticks="outside",
            tickmode="array",
            tickvals=tick_vals[feature],
            ticktext=tick_text[feature],
            outlinecolor="rgb(0.0, 0.0, 0.0)",
            outlinewidth=1,
        )
    )
    
    dot_size = 5 if data_source == "VCSN" else 20

    fig.update_traces(marker={
        'size': dot_size,
        "symbol": "circle"
    })

    fig.show()
    
    return fig

In [ ]:
vcsn_wet_fig = build_day_plot(df_vcsn_grouped, "wet", "VCSN")
vcsn_wet_fig.write_image(f"{DIR_PLOTS}/VCSN_fewest_wet_days.png", format="png", scale=2)

vcsn_dry_fig = build_day_plot(df_vcsn_grouped, "dry", "VCSN")
vcsn_dry_fig.write_image(f"{DIR_PLOTS}/VCSN_most_dry_days.png", format="png", scale=2)

In [ ]:
feature = "dry"
nums = tick_vals[feature]

val_max = df_vcsn_grouped[feature].max()
print(f"Max: {val_max}")
val_min = df_vcsn_grouped[feature].min()
print(f"Min: {val_min}")
print()

for num in nums:
    print(f"{((num - val_min) / (val_max - val_min)):.3f}")

## For thesis - median

In [ ]:
group_cols = [
    "region",
    "grid_cell",
]

df_wah = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_PrecipPercent")
df_wah = df_wah.loc[df_wah["source"] == "ALL"].reset_index(drop=True)

df_wah_grouped = df_wah[group_cols + ["wet", "dry"]].groupby(group_cols).agg({"wet": "median", "dry": "median"}).reset_index(drop=False)

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_wah_grouped['grid_cell'].str.split('_', n=2, expand=True)
split_df.columns = ["lat", "lon"]
split_df["lat"] = split_df["lat"].astype(float)
split_df["lon"] = split_df["lon"].astype(float)
df_wah_grouped = pd.merge(df_wah_grouped, split_df, how="left", left_index=True, right_index=True)

df_wah_grouped["wet"] = np.floor(df_wah_grouped["wet"])
df_wah_grouped["dry"] = np.floor(df_wah_grouped["dry"])



df_vcsn = pd.read_parquet(f"{DIR_VCSN}/RVI/Chunks_Fixed_PrecipPercent/")
df_vcsn = df_vcsn.loc[df_vcsn["year"].astype(int).between(2007, 2016)]

df_vcsn_grouped = df_vcsn[group_cols + ["wet", "dry"]].groupby(group_cols).agg({"wet": "median", "dry": "median"}).reset_index(drop=False)

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_vcsn_grouped['grid_cell'].str.split('_', n=2, expand=True)
split_df.columns = ["lat", "lon"]
split_df["lat"] = split_df["lat"].astype(float)
split_df["lon"] = split_df["lon"].astype(float)
df_vcsn_grouped = pd.merge(df_vcsn_grouped, split_df, how="left", left_index=True, right_index=True)

df_vcsn_grouped["wet"] = np.floor(df_vcsn_grouped["wet"])
df_vcsn_grouped["dry"] = np.floor(df_vcsn_grouped["dry"])

In [ ]:
def build_day_plot(df, feature, data_source):
    colour_range = {
        "wet": (2, 14),
        "dry": (270, 340)
    }
    
    colour_scale = {
        "wet": "agsunset",
        "dry": "oranges"
    }
    
    
    fig = px.scatter(
        df, 
        x='lon', 
        y='lat', 
        height=950, 
        width=800, 
        color=feature,
        template="plotly_white",
        color_continuous_scale=colour_scale[feature],
        range_color=colour_range[feature]
    )
    
    amount_str = "Fewest" if feature == "wet" else "Most"
    d_ticks = 2 if feature == "wet" else 10
    
    fig.update_layout(
        title=f"{amount_str} days needed to reach 25% of annual rainfall - {data_source}",
        coloraxis_colorbar=dict(
            title="",
            orientation="h",
            y=-0.2,
            yanchor="bottom",
            dtick=d_ticks,
            ticks="outside",
            outlinecolor="rgb(0.0, 0.0, 0.0)",
            outlinewidth=1,
        )
    )
    
    dot_size = 5 if data_source == "VCSN" else 20

    fig.update_traces(marker={
        'size': dot_size,
        "symbol": "circle"
    })

    fig.show()
    
    return fig

In [ ]:
fig_vcsn_wet = build_day_plot(df_vcsn_grouped, "wet", "VCSN")
fig_vcsn_wet.write_image(f"{DIR_PLOTS_THESIS}/VCSN_fewest_wet_days.png", format="png", scale=2)

fig_vcsn_dry = build_day_plot(df_vcsn_grouped, "dry", "VCSN")
fig_vcsn_dry.write_image(f"{DIR_PLOTS_THESIS}/VCSN_most_dry_days.png", format="png", scale=2)

In [ ]:
fig_wah_wet = build_day_plot(df_wah_grouped, "wet", "WaH ALL")
fig_wah_wet.write_image(f"{DIR_PLOTS_THESIS}/WaH_fewest_wet_days.png", format="png", scale=2)

fig_wah_dry = build_day_plot(df_wah_grouped, "dry", "WaH ALL")
fig_wah_dry.write_image(f"{DIR_PLOTS_THESIS}/WaH_most_dry_days.png", format="png", scale=2)

## For Thesis - Range

In [ ]:
group_cols = [
    "region",
    "grid_cell",
]

df_wah = pd.read_parquet(f"{DIR_HAPPI}/RVI/Chunks_Fixed_PrecipPercent")
df_wah = df_wah.loc[df_wah["source"] == "ALL"].reset_index(drop=True)

df_wah_grouped = df_wah[group_cols + ["wet", "dry"]].groupby(group_cols).agg({"wet": ["median", "max", "min"], "dry": ["median", "max", "min"]}).reset_index(drop=False)
df_wah_grouped.columns = ["region", "grid_cell", "wet_median", "wet_max", "wet_min", "dry_median", "dry_max", "dry_min"]

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_wah_grouped['grid_cell'].str.split('_', n=2, expand=True)
split_df.columns = ["lat", "lon"]
split_df["lat"] = split_df["lat"].astype(float)
split_df["lon"] = split_df["lon"].astype(float)
df_wah_grouped = pd.merge(df_wah_grouped, split_df, how="left", left_index=True, right_index=True)

df_wah_grouped["wet_median"] = np.floor(df_wah_grouped["wet_median"])
df_wah_grouped["dry_median"] = np.floor(df_wah_grouped["dry_median"])
df_wah_grouped["wet_range"] = df_wah_grouped["wet_max"] - df_wah_grouped["wet_min"]
df_wah_grouped["dry_range"] = df_wah_grouped["dry_max"] - df_wah_grouped["dry_min"]



df_vcsn = pd.read_parquet(f"{DIR_VCSN}/RVI/Chunks_Fixed_PrecipPercent/")
df_vcsn = df_vcsn.loc[df_vcsn["year"].astype(int).between(2007, 2016)]

df_vcsn_grouped = df_vcsn[group_cols + ["wet", "dry"]].groupby(group_cols).agg({"wet": ["median", "max", "min"], "dry": ["median", "max", "min"]}).reset_index(drop=False)
df_vcsn_grouped.columns = ["region", "grid_cell", "wet_median", "wet_max", "wet_min", "dry_median", "dry_max", "dry_min"]

# Step: Manipulate strings of 'grid_cell' and perform a split on '_'
split_df = df_vcsn_grouped['grid_cell'].str.split('_', n=2, expand=True)
split_df.columns = ["lat", "lon"]
split_df["lat"] = split_df["lat"].astype(float)
split_df["lon"] = split_df["lon"].astype(float)
df_vcsn_grouped = pd.merge(df_vcsn_grouped, split_df, how="left", left_index=True, right_index=True)

df_vcsn_grouped["wet_median"] = np.floor(df_vcsn_grouped["wet_median"])
df_vcsn_grouped["dry_median"] = np.floor(df_vcsn_grouped["dry_median"])
df_vcsn_grouped["wet_range"] = df_vcsn_grouped["wet_max"] - df_vcsn_grouped["wet_min"]
df_vcsn_grouped["dry_range"] = df_vcsn_grouped["dry_max"] - df_vcsn_grouped["dry_min"]

In [ ]:
def build_day_plot(df, feature, data_source):
    colour_range = {
        "wet": (0, 14),
        "dry": (0, 50)
    }
    
    colour_scale = {
        "wet": "agsunset",
        "dry": "oranges"
    }
    
    
    fig = px.scatter(
        df, 
        x='lon', 
        y='lat', 
        height=950, 
        width=800, 
        color=f"{feature}_range",
        template="plotly_white",
        color_continuous_scale=colour_scale[feature],
        range_color=colour_range[feature]
    )
    
    d_ticks = 2 if feature == "wet" else 10
    
    fig.update_layout(
        title=f"Range of {feature} days needed to reach 25% of annual rainfall - {data_source}",
        coloraxis_colorbar=dict(
            title="",
            orientation="h",
            y=-0.2,
            yanchor="bottom",
            dtick=d_ticks,
            ticks="outside",
            outlinecolor="rgb(0.0, 0.0, 0.0)",
            outlinewidth=1,
        )
    )
    
    dot_size = 5 if data_source == "VCSN" else 20

    fig.update_traces(marker={
        'size': dot_size,
        "symbol": "circle"
    })

    fig.show()
    
    return fig

In [ ]:
fig_vcsn_wet = build_day_plot(df_vcsn_grouped, "wet", "VCSN")
fig_vcsn_wet.write_image(f"{DIR_PLOTS_THESIS}/VCSN_range_wet_days.png", format="png", scale=2)

fig_vcsn_dry = build_day_plot(df_vcsn_grouped, "dry", "VCSN")
fig_vcsn_dry.write_image(f"{DIR_PLOTS_THESIS}/VCSN_range_dry_days.png", format="png", scale=2)

In [ ]:
fig_wah_wet = build_day_plot(df_wah_grouped, "wet", "WaH ALL")
fig_wah_wet.write_image(f"{DIR_PLOTS_THESIS}/WaH_range_wet_days.png", format="png", scale=2)

fig_wah_dry = build_day_plot(df_wah_grouped, "dry", "WaH ALL")
fig_wah_dry.write_image(f"{DIR_PLOTS_THESIS}/WaH_range_dry_days.png", format="png", scale=2)

# Placeholder